In [ ]:
import caiman as cm
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
import tifffile as tif
from caiman.source_extraction.cnmf.cnmf import load_CNMF
from caiman.base.rois import register_multisession

from scipy.io import loadmat

from plotnine import ggplot, geom_density, geom_histogram, aes, geom_jitter, geom_errorbar, stat_smooth,facet_grid, stat_summary,theme, geom_vline, facet_wrap, geom_point, geom_line, guides, guide_legend, ggtitle, xlab, ylab
from plotnine.themes import theme


import pandas as pd

import bokeh.plotting as bpl
bpl.output_notebook()

In [ ]:
# file structure


dims = (512,512)

group_num = 1

# base_dir = '/gpfs/data/shohamlab/shared_data/yi_recordings/yi_new_holder_results/line3/new_experiments_201911/11132019/mouse9'
# groups = ['region1', 'region2']
# group_filedirs = ['region1', 'region2']
# out_pattern = "*_mc"
# ref_files = ['region1/ref.tif', 'region2/ref.tif']

# base_dir = '/gpfs/data/shohamlab/shared_data/yi_recordings/yi_new_holder_results/sst'
# groups=['mouse7', 'mouse8']
# group_filedirs = ['mouse7_single', 'mouse8_single']
# out_pattern = "*_mc.tif"
# ref_files = ['mouse7_single/ref.tif', 'mouse8_single/ref.tif']

base_dir = '/gpfs/data/shohamlab/shared_data/yi_recordings/yi_new_holder_results/line3/new_experiments_201911/11082019/mouse6/'
groups = ['region1', 'region2']
group_filedirs = ['region1', 'region2']
out_pattern = "*_mc"
ref_files = ['region1/ref.tif', 'region2/ref.tif']


data_dict = {}
os.chdir(base_dir)

import re
def parse_rec_name(rec_name):
    base_rec_name = os.path.basename(rec_name)
    p_mpa = re.compile('[0-9]*MPA')
    mpa_tag = float('.' + p_mpa.findall(base_rec_name)[0][:-3])*10
    p_dc = re.compile('[0-9]*DC')
    dc_tag = float('.' + p_dc.findall(base_rec_name)[0][:-2])
    return (mpa_tag, dc_tag)






class Struct():
    def __init__(self):
        return




# construct data dict object keyed on groups
for i,grpname in enumerate(groups):
    ref_img = tif.imread(ref_files[i])
    #sort by video number
    group_fnames = sorted(list(glob.glob(os.path.join(group_filedirs[i], out_pattern))), key = lambda x: int(x.split('-')[-1].split('_')[0]))
    
    tmpstruct = Struct()
    tmpstruct.ref_img = ref_img
    tmpstruct.cnmf_list = []
    tmpstruct.name_list = group_fnames
    tmpstruct.mpa_list = [parse_rec_name(x)[0] for x in group_fnames]
    tmpstruct.dc_list = [parse_rec_name(x)[1] for x in group_fnames]
    
    #for k, out_name in enumerate(tmpstruct.name_list):
    #    tmpstruct.cnmf_list += [load_CNMF(group_fnames[k])]
        
    data_dict[grpname] = tmpstruct
    
    

group_name = groups[group_num]
group_data = data_dict[group_name]
group_dir = group_filedirs[group_num]

plt.figure(figsize=(5,5))
plt.title(group_name)
plt.imshow(group_data.ref_img)
plt.show()


# Combine file plots

In [ ]:
dims = (512,512)

n_trials = 16
trial_frames = 250
stim_frame=50
fps = 10
pre_stim = 30; post_stim = 60

group_files = ['C:/Users/bnste/Downloads/amy_us_results/caiman_out_10fps_p0.hdf5']
ref_files = ['C:/Users/bnste/Downloads/amy_us_results/ref_green.tif']
roi_file = 'C:/Users/bnste/Downloads/amy_us_results/SST131_sendBen.mat'
roi_mat = loadmat(roi_file)

mouse_list = ['mouse0']
region_list = ['region0']
mpa_list = [[.01,.04,.06,.08,.08,.08,.08,.08,.08,0]]
dc_list = [[.5,.5 ,.5 ,.1, .2 ,.3 ,.4, .5, .05, .5]]


manual_masks = roi_mat['ROI_masks_all']


In [ ]:
fig = plt.figure(figsize=(5,5))
print(manual_masks.shape)
plt.imshow(manual_masks.max(axis=-1))
plt.show()

In [ ]:
comb_cnmf = load_CNMF(group_files[0])
ref_img = tif.imread(ref_files[0])

cell_idx = comb_cnmf.estimates.idx_components
A = comb_cnmf.estimates.A[:,cell_idx]
b = comb_cnmf.estimates.b
C = comb_cnmf.estimates.C[cell_idx]
f = comb_cnmf.estimates.f
S = comb_cnmf.estimates.S[cell_idx]
F_dff = comb_cnmf.estimates.F_dff[cell_idx]
snr = comb_cnmf.estimates.SNR_comp[cell_idx]

strong_idx = (snr>3).nonzero()[0]
print('Using {} strong components'.format(len(strong_idx)))
fig = plt.figure(figsize=(5,5))
plt.imshow(A[:,strong_idx].sum(axis=-1).reshape(dims,order='F'))
plt.show()

comb_cnmf.estimates.dims = comb_cnmf.dims
comb_cnmf.estimates.nb_view_components(img=ref_img, idx = comb_cnmf.estimates.idx_components)

In [ ]:
mask_score = np.tensordot(manual_masks.reshape((-1,manual_masks.shape[-1]), order='F'),np.power(A[:,strong_idx].todense(),2) , axes=(0,0))
max_match = np.argmax(mask_score,axis=0)
max_score = mask_score[max_match,np.arange(mask_score.shape[1])]
selected_rois = strong_idx[(max_score > .2).nonzero()]
selected_rois_manual = max_match[(max_score > .2).nonzero()] 

In [ ]:
def get_file_table(h5file, mouse,region,mpa_list_single, dc_list_single, n_trials, trial_frames,stim_frame,
                  pre_stim, post_stim):
    print(h5file, mouse,region,mpa_list_single, dc_list_single)
    comb_cnmf = load_CNMF(h5file)
    
        
    thresh_low,thresh_high = [5,90]; # percentile thresholds
    thresh_mid = [30,50] # non responder thresholds
    
    
    
    cell_idx = comb_cnmf.estimates.idx_components #filter to include only these cells
    print('Using {}/{} components'.format(len(cell_idx), len(comb_cnmf.estimates.C)))
    
    
    A = comb_cnmf.estimates.A[:,cell_idx]
    b = comb_cnmf.estimates.b
    C = comb_cnmf.estimates.C[cell_idx]
    f = comb_cnmf.estimates.f
    S = comb_cnmf.estimates.S[cell_idx]
    F_dff = comb_cnmf.estimates.F_dff[cell_idx]

    

    

    
    prestim = slice(stim_frame-pre_stim,stim_frame) 
    poststim = slice(stim_frame+1, stim_frame+post_stim)

     
    snr_tab = pd.DataFrame({'roi':cell_idx,'snr': comb_cnmf.estimates.SNR_comp[cell_idx]})
        
        
    trial_reshaped_dff = F_dff.reshape((F_dff.shape[0],len(mpa_list_single),-1,trial_frames), order='C')[:,:,1:-1,:]
    trial_mean_dff =F_dff.reshape((F_dff.shape[0],len(mpa_list_single),-1,trial_frames), order='C')[:,:,1:-1,:].mean(axis=2)
    trial_mean_S = S.reshape((S.shape[0],len(mpa_list_single),-1,trial_frames), order='C')[:,:,1:-1,:].mean(axis=2)

    
    
    
    dff_change = F_dff.reshape((F_dff.shape[0],len(mpa_list_single),-1,trial_frames),order='C')[:,:,1:-1,:]
    dff_immediate_stim = dff_change[:,:,:,stim_frame] - dff_change[:,:,:,prestim].mean(axis=-1)
    dff_change = dff_change[:,:,:,poststim].mean(axis=-1) - dff_change[:,:,:,prestim].mean(axis=-1)
    dff_change_mean = dff_change.mean(axis=(1,2))

    
#     # use this section to categorize responders by percentile
#     pct_low, pct_high = np.percentile(dff_change_mean,(thresh_low,thresh_high))
#     pct_mid_low, pct_mid_high = np.percentile(dff_change_mean,thresh_mid)
#     responders = (dff_change_mean>pct_high).nonzero()
#     non_responders =   ((dff_change_mean<= pct_high) & (dff_change_mean>pct_low)).nonzero()
#     neg_responders = (dff_change_mean<= pct_low).nonzero()
    
    
    # use this section to categorize responders by absolute df/f
    low_cutoff = -.01
    high_cutoff = .01
    weak_cutoffs = [-.01 ,.01]
    responders = (dff_change_mean>high_cutoff).nonzero()
    non_responders =   ((dff_change_mean<= weak_cutoffs[1]) & (dff_change_mean>weak_cutoffs[0])).nonzero()
    neg_responders = (dff_change_mean<= low_cutoff).nonzero()
    
    
    # add background as roi with response type 2
    bg_ind = dff_change.shape[0]
    tot_bg = b.sum(axis=0).dot(f).reshape((1,len(mpa_list_single),-1,trial_frames),order='C')[:,:,1:-1,:]
    tot_bg_change = (tot_bg[:,:,:,poststim].mean(axis=-1) - tot_bg[:,:,:,prestim].mean(axis=-1)) / tot_bg[:,:,:,prestim].mean(axis=-1)
    immediate_stim_bg = (tot_bg[:,:,:,stim_frame] - tot_bg[:,:,:,prestim].mean(axis=-1)) / tot_bg[:,:,:,prestim].mean(axis=-1)
    dff_change = np.concatenate([dff_change, tot_bg_change],axis=0)
    dff_immediate_stim = np.concatenate([dff_immediate_stim, immediate_stim_bg],axis=0)
    
    # transform dff change matrix and associated cell/mov/trial indices into table-based column format
    ind_array = np.indices(dff_change.shape) # cell_inds, mov_inds, trial_inds
    dff_change_tabformat = dff_change.reshape((-1,1), order='C')
    dff_immediate_stim_tabformat = dff_immediate_stim.reshape((-1,1), order='C')
    cell_inds, mov_inds, trial_inds = ind_array.reshape( (ind_array.shape[0],-1), order='C' )
    mpa_tabformat = np.array(mpa_list_single)[mov_inds]
    dc_tabformat = np.array(dc_list_single)[mov_inds]
    
    bg_inds = (cell_inds == bg_ind).nonzero()
    cell_inds[bg_inds] = 0
    roi_labels = cell_idx[cell_inds]
    roi_labels[bg_inds] = -1
    
    tab = pd.DataFrame({
                        'mov': mov_inds,
                        'roi': roi_labels,
                        'trial': trial_inds,
                        'dff_resp': dff_change.flatten(),
                        'immediate_resp': dff_immediate_stim.flatten(),
                        'mpa': mpa_tabformat,
                        'dc':dc_tabformat,
                        'resp_type': np.isin(cell_inds, responders).astype('int') - np.isin(cell_inds, neg_responders).astype('int')
                        })
    
    tab.loc[tab.roi==-1,'resp_type']=2
    tab['mouse_reg'] = pd.Categorical( ['_'.join([mouse,region])]*len(cell_inds))
    tab.loc[(~np.isin(cell_inds,non_responders)) & (tab.resp_type==0),'resp_type'] = 10
    tab = tab.join(snr_tab.set_index('roi'),on=['roi'])
    
    
    # trial trace table
    
    dff_change = F_dff.reshape((F_dff.shape[0],len(mpa_list_single),-1,trial_frames),order='C')[:,:,1:-1,:]
    dff_change = dff_change - dff_change[:,:,:,prestim].mean(axis=-1, keepdims=True)
    tot_bg = b.sum(axis=0).dot(f).reshape((1,len(mpa_list_single),-1,trial_frames),order='C')[:,:,1:-1,:]
    tot_bg_change = (tot_bg - tot_bg[:,:,:,prestim].mean(axis=-1, keepdims=True)) / tot_bg[:,:,:,prestim].mean(axis=-1, keepdims=True)
    dff_change = np.concatenate([dff_change, tot_bg_change],axis=0)
    ind_array = np.indices(dff_change.shape) # cell_inds, mov_inds, trial_inds, frame_inds
    dff_change_tabformat = dff_change.reshape((-1,1), order='C')
    cell_inds, mov_inds, trial_inds, frame_inds = ind_array.reshape( (ind_array.shape[0],-1), order='C' )
    mpa_tabformat = np.array(mpa_list_single)[mov_inds]
    dc_tabformat = np.array(dc_list_single)[mov_inds]
    
    
    bg_inds = (cell_inds == bg_ind).nonzero()
    cell_inds[bg_inds] = 0
    roi_labels = cell_idx[cell_inds]
    roi_labels[bg_inds] = -1
    
    
    print('dff_change.shape: {}'.format(dff_change.shape))
    
    trial_tab = pd.DataFrame({
                        'mov': mov_inds,
                        'roi': roi_labels,
                        'trial': trial_inds,
                        'frame': frame_inds,
                        'dff_resp': dff_change.flatten(),
                        'mpa': mpa_tabformat,
                        'dc':dc_tabformat,
                        'resp_type': np.isin(cell_inds, responders).astype('int') - np.isin(cell_inds, neg_responders).astype('int')
                        })
    
    trial_tab['mouse_reg'] = pd.Categorical( ['_'.join([mouse,region])]*len(cell_inds))
    trial_tab.loc[trial_tab.roi==-1,'resp_type']=2
    trial_tab.loc[(~np.isin(cell_inds,non_responders)) & (trial_tab.resp_type==0),'resp_type'] = 10
    trial_tab = trial_tab.join(snr_tab.set_index('roi'),on=['roi'])
    
    
    return tab, trial_tab




In [ ]:
# filter out rois with outlier responses
dff_max_thresh = np.inf
dff_min_thresh = -np.inf

# filter out rois based on snr
snr_thresh = 3

n = len(mouse_list)
res_list = [ get_file_table(*x) for x in list(zip(group_files, mouse_list, region_list, mpa_list, dc_list,
                                                 n*[n_trials],n*[trial_frames],n*[stim_frame],n*[pre_stim], n*[post_stim]))[:]]
res_tab = pd.concat([x[0] for x in res_list])
res_trial_tab = pd.concat([x[1] for x in res_list])

In [ ]:
good_rois = res_trial_tab.groupby(['mouse_reg','roi'])['resp_type','dff_resp','snr'].agg(max_dff=pd.NamedAgg(column='dff_resp', aggfunc='max'),min_dff=pd.NamedAgg(column='dff_resp', aggfunc='min'),resp_type=pd.NamedAgg(column='resp_type', aggfunc='first') , snr=pd.NamedAgg(column='snr', aggfunc='first') )
good_rois = good_rois[((good_rois.max_dff <dff_max_thresh) &(good_rois.min_dff >dff_min_thresh) & (good_rois.snr > snr_thresh)) | (good_rois.resp_type==2)  ]
good_rois = good_rois.drop(columns=['resp_type', 'snr'])
print('{} good ROIs'.format(len(good_rois)))

res_tab = res_tab.join(good_rois,how='inner', on=['mouse_reg','roi'])
res_trial_tab = res_trial_tab.join(good_rois,how='inner', on=['mouse_reg','roi'])

resp_vs = res_tab.groupby(['mpa','dc','resp_type','mouse_reg'], as_index=False, observed=True).agg({'dff_resp':['mean', 'std','count']})
resp_vs.columns = ['mpa', 'dc', 'resp_type', 'mouse_reg', 'dff_resp', 'dff_std', 'nsamples']
print(resp_vs)

trial_trace_vs = res_trial_tab.groupby(['mpa','dc','resp_type','mouse_reg','frame'], as_index=False, observed=True).agg({'dff_resp':['mean', 'std','count']})
trial_trace_vs.columns = ['mpa', 'dc', 'resp_type', 'mouse_reg', 'frame','dff_resp', 'dff_std', 'nsamples']
print(trial_trace_vs)

cellstats = res_tab.groupby(['mpa','dc','resp_type','mouse_reg','roi'], as_index=False, observed=True).agg({'dff_resp':['mean', 'std','count']})
cellstats.columns = ['mpa', 'dc', 'resp_type', 'mouse_reg', 'roi', 'dff_resp', 'dff_std', 'nsamples']
cellstats['dff_stderr'] = cellstats.dff_std / np.sqrt(cellstats.nsamples)
print(cellstats)

In [ ]:
data_overall = res_tab.copy()
data=data_overall.groupby(['mpa','dc','resp_type','mouse_reg'],as_index=False)['dff_resp'].mean().reset_index()

data_sem = (pd.DataFrame(data_overall.groupby(['mpa','dc','resp_type','mouse_reg'])['dff_resp'].std())) / np.sqrt((pd.DataFrame(data_overall.groupby(['mpa','dc','resp_type','mouse_reg'])['dff_resp'].count())))
data_sem = data_sem.reset_index()
data['dff_sem'] = data_sem.dff_resp
data['min_conf'] = data.dff_resp - data.dff_sem
data['max_conf'] = data.dff_resp + data.dff_sem

def label_func(resp_type):
    resp_dict={
        '1':'Positive',
        '0':'Weak',
        '-1':'Negative',
        '2': 'Background',
        '10': 'Other'}
    return resp_dict[resp_type]

plot= ( ggplot(data[data.dc==.50],aes(x='mpa',y="dff_resp", group='mouse_reg',color='mouse_reg')) 
    + ggtitle('DF/F response vs US intensity by responder type. DC=50%')
    + xlab('Intensity (MPA)')
    + ylab('Post-stim Change in DF/F')
    + theme(aspect_ratio=2.5)
    + geom_errorbar(aes(ymin='min_conf', ymax='max_conf'), width=.005)
    + facet_wrap('~resp_type', labeller=label_func, nrow=1)
    + geom_line(stat='summary')
    + geom_line(data=data_overall[data_overall.dc==.50], group=1,color='red',size=1,linetype='--', stat='summary')
    + guides(color=guide_legend(title='Mouse / region')))


fig=plot.draw();
fig.set_size_inches((12,4))

In [ ]:
data_overall = res_tab.copy()
data=data_overall.groupby(['mpa','dc','resp_type','mouse_reg'],as_index=False)['dff_resp'].mean().reset_index()

data_sem = (pd.DataFrame(data_overall.groupby(['mpa','dc','resp_type','mouse_reg'])['dff_resp'].std())) / np.sqrt((pd.DataFrame(data_overall.groupby(['mpa','dc','resp_type','mouse_reg'])['dff_resp'].count())))
data_sem = data_sem.reset_index()
data['dff_sem'] = data_sem.dff_resp
data['min_conf'] = data.dff_resp - data.dff_sem
data['max_conf'] = data.dff_resp + data.dff_sem

def label_func(resp_type):
    resp_dict={
        '1':'Positive',
        '0':'Weak',
        '-1':'Negative',
        '2': 'Background',
        '10': 'Other'}
    return resp_dict[resp_type]

plot= ( ggplot(data[data.mpa==data.mpa.max()],aes(x='dc',y="dff_resp", group='mouse_reg',color='mouse_reg')) 
    + ggtitle('DF/F response vs US Duty Cycle (DC) by responder type. Pressure = {}MPA'.format(data.mpa.max()))
    + xlab('Duty Cycle (%)')
    + ylab('Post-stim Change in DF/F')
    + theme(aspect_ratio=2.5)
    + geom_errorbar(aes(ymin='min_conf', ymax='max_conf'), width=.05)
    + facet_wrap('~resp_type', labeller=label_func, nrow=1)
    + geom_line(stat='summary')
    + geom_line(data=data_overall[data_overall.mpa == data_overall.mpa.max()], group=1,color='red',size=1,linetype='--', stat='summary')
    + guides(color=guide_legend(title='Mouse / region')))


fig=plot.draw();
fig.set_size_inches((12,4))

In [ ]:
# trial time traces vs intensity by responder type

def label_func_grid(inp):
    return ('_'.join(inp[0].split('_')[:2]) ,label_func(inp[1]))

data=res_trial_tab[(res_trial_tab.dc==.5)].groupby(['mouse_reg', 'mpa','dc','roi','resp_type','frame'])['dff_resp'].mean().reset_index()
data['mpa'] = pd.Categorical(data.mpa)
data['rel_time'] =  data.frame / fps # time in seconds
data=data.sort_values('rel_time')


data_by_mpa = pd.DataFrame(data.groupby(['mpa','dc','resp_type','rel_time','mouse_reg'],as_index=False)['dff_resp'].mean()).reset_index()
#data_overall = data.groupby(['dc','resp_type','rel_time'],as_index=False)['dff_resp'].mean()


plot= ( ggplot(data_by_mpa,aes(x='rel_time',y='dff_resp' ,group='mpa',color='mpa'))
    + ggtitle('trial DF/F response vs US intensity by responder type. DC=50%')
    + xlab('Time since trial start (s)')
    + ylab('Post-stim Change in DF/F')
    + theme(panel_spacing=.01)
    + facet_grid('mouse_reg~resp_type')
    #+ stat_summary(group='mpa' ,geom = 'line', size=.6)
    + geom_line(size=.6,mapping=aes(group='mpa',color='mpa'), stat='identity')
    #+ stat_smooth(size=.6, method='mavg', method_args={'window':1})
    + guides(color=guide_legend(title='Intensity (MPA)'))
    + geom_vline(xintercept=stim_frame / fps, alpha=.3)
      )
fig=plot.draw();
fig.set_size_inches((10,2.5*len(data.mouse_reg.unique())))

In [ ]:
with pd.option_context('display.precision', 2):
    resp_counts = res_tab[['mouse_reg','resp_type','roi']].drop_duplicates().groupby(['mouse_reg','resp_type'],as_index=False)['roi'].count()
    resp_counts = pd.DataFrame(resp_counts).reset_index()
    
    resp_counts['frac'] = resp_counts.roi / resp_counts[resp_counts.resp_type !=2].groupby(['mouse_reg'])['roi'].transform('sum')
    resp_counts = resp_counts.rename({'roi':'num_rois'}, axis=1).sort_values(['mouse_reg','resp_type'])
    df=resp_counts.style
df

In [ ]:
plt.hist(res_tab.groupby('roi')['dff_resp'].mean().values,30)

In [ ]:
# stim responses vs intensity by responder type

mean_type = 'all' # 'weighted' or 'all'



max_norm = False

data=resp_vs[(resp_vs.dc==.5) ].copy()
data['dff_abs'] = np.abs(data.dff_resp)



if max_norm:
    data.dff_resp = data.dff_resp / data.groupby(['dc','resp_type','mouse_reg'])['dff_abs'].transform('max')

if mean_type == 'all':
    data_overall=data.groupby(['mpa','dc','resp_type'],as_index=False)['dff_resp'].mean()
elif mean_type == 'weighted':
    data_overall = data.copy()
    data_overall['dff_resp'] = (data_overall['dff_resp'] * data_overall['nsamples']) / data_overall.groupby(['mpa','dc','resp_type'],as_index=False)['nsamples'].transform('sum')['nsamples']
    data_overall=data_overall.groupby(['mpa','dc','resp_type'],as_index=False)['dff_resp'].sum()
else:
    raise Exception("'mean_type' must be 'weighted' or 'all'")

plot= ( ggplot(data,aes(x='mpa',y="dff_resp", group='mouse_reg',color='mouse_reg')) 
    + ggtitle('DF/F response vs US intensity by responder type. DC=50%')
    + xlab('Intensity (MPA)')
    + ylab('Post-stim Change in DF/F')
    + theme(aspect_ratio=2.5)
    + facet_wrap('~resp_type', labeller=label_func, nrow=1)
    + geom_line()
    + geom_line(data=data_overall, group=1,color='red',size=1,linetype='--')
    + guides(color=guide_legend(title='Mouse / region')))

fig=plot.draw();
fig.set_size_inches((12,4))

In [ ]:
# stim responses vs Intensity by responder type




data=resp_vs[(resp_vs.mpa==resp_vs.mpa.max())].copy()

data['dff_abs'] = np.abs(data.dff_resp)

if max_norm:
    data.dff_resp = data.dff_resp / data.groupby(['mpa','resp_type','mouse_reg'])['dff_abs'].transform('max')

if mean_type == 'all':
    data_overall=data.groupby(['mpa','dc','resp_type'],as_index=False)['dff_resp'].mean()
elif mean_type == 'weighted':
    data_overall = data.copy()
    data_overall['dff_resp'] = (data_overall['dff_resp'] * data_overall['nsamples']) / data_overall.groupby(['mpa','dc','resp_type'],as_index=False)['nsamples'].transform('sum')['nsamples']
    data_overall=data_overall.groupby(['mpa','dc','resp_type'],as_index=False)['dff_resp'].sum()
else:
    raise Exception("'mean_type' must be 'weighted' or 'all'")

plot=( ggplot(data,aes(x='dc',y="dff_resp", group='mouse_reg', color='mouse_reg'))
    + ggtitle('DF/F response vs US Duty Cycle (DC) by responder type. I=0.8MPA')
    + xlab('DC')
    + ylab('Post-stim Change in DF/F')
    + theme(aspect_ratio=2.5)
    + facet_wrap('~resp_type', labeller=label_func, nrow=1)
    + geom_line()
    + geom_line(data=data_overall, group=1,color='red',size=1,linetype='--')
    #+ stat_smooth(mapping=aes(x='dc',y='dff_resp'), color='red', method='lm', size=1)
    + guides(color=guide_legend(title='Mouse / region')))

fig=plot.draw();
fig.set_size_inches((10,4))

In [ ]:
# trial time traces vs intensity by responder type

def label_func_grid(inp):
    return ('_'.join(inp[0].split('_')[:2]) ,label_func(inp[1]))

data=trial_trace_vs[(trial_trace_vs.dc==.5)].copy()
data['mpa'] = pd.Categorical(data.mpa)
data['rel_time'] =  data.frame / 3.5 # time in seconds
data=data.sort_values('rel_time')


data_by_mpa = pd.DataFrame(data.groupby(['mpa','dc','resp_type','rel_time','mouse_reg'],as_index=False)['dff_resp'].mean()).reset_index()
#data_overall = data.groupby(['dc','resp_type','rel_time'],as_index=False)['dff_resp'].mean()


plot= ( ggplot(data_by_mpa,aes(x='rel_time',y='dff_resp' ,group='mpa',color='mpa'))
    + ggtitle('trial DF/F response vs US intensity by responder type. DC=50%')
    + xlab('Time since trial start (s)')
    + ylab('Post-stim Change in DF/F')
    + theme(panel_spacing=.01)
    + facet_grid('mouse_reg~resp_type')
    #+ stat_summary(group='mpa' ,geom = 'line', size=.6)
    + geom_line(size=.6,mapping=aes(group='mpa',color='mpa'), stat='identity')
    #+ stat_smooth(size=.6, method='mavg', method_args={'window':1})
    + guides(color=guide_legend(title='Intensity (MPA)'))
    + geom_vline(xintercept=stim_frame / 3.5, alpha=.3)
      )
fig=plot.draw();
fig.set_size_inches((10,2.5*len(data.mouse_reg.unique())))

In [ ]:
# trial time traces vs DC by responder type



data=trial_trace_vs[(trial_trace_vs.mpa==trial_trace_vs.mpa.max())].copy()
data['rel_time'] =  data.frame / 3.5 # time in seconds
data['dc'] = pd.Categorical(data.dc)

data_by_dc = pd.DataFrame(data.groupby(['mpa','dc','resp_type','rel_time','mouse_reg'],as_index=False)['dff_resp'].mean()).reset_index()
#data_overall = data.groupby(['mpa','resp_type','rel_time'],as_index=False)['dff_resp'].mean()


plot= ( ggplot(data_by_dc,aes(x='rel_time',y='dff_resp' ,group='dc',color='dc'))
    + ggtitle('trial DF/F response vs US Duty Cycle (DC) by responder type. I={:.2f}MPA'.format(trial_trace_vs.mpa.max()))
    + xlab('Time since trial start (s)')
    + ylab('Post-stim Change in DF/F')
    + theme(panel_spacing=.01)
    + facet_grid('mouse_reg~resp_type')
    #+ stat_summary(group='mpa' ,geom = 'line', size=.6)
    + geom_line(size=.6,mapping=aes(group='dc',color='dc'), stat='identity')
    #+ stat_smooth(size=.6, method='mavg', method_args={'window':1})
    + guides(color=guide_legend(title='Duty Cycle'))
    + geom_vline(xintercept=stim_frame / 3.5, alpha=.3)
      )
fig=plot.draw();
fig.set_size_inches((10,2.5*len(data.mouse_reg.unique())))

In [ ]:
data=res_tab
data = data[(data.mpa==data.mpa.max())]
data_roi = pd.DataFrame(data.groupby(['trial','resp_type','roi','mouse_reg'])['dff_resp'].mean()).reset_index()
data_mean = pd.DataFrame(data_roi.groupby(['trial','resp_type','mouse_reg'])['dff_resp'].mean()).reset_index()
data_sem = (pd.DataFrame(data_roi.groupby(['trial','resp_type','mouse_reg'])['dff_resp'].std())) / np.sqrt((pd.DataFrame(data_roi.groupby(['trial','resp_type','mouse_reg'])['dff_resp'].count())))
data_sem = data_sem.reset_index()
data_mean['sem'] = data_sem.dff_resp


plot=( ggplot(data_roi,aes(x='trial',y="dff_resp"))
    + ylab('Post-stim Change in DF/F')
    + theme(aspect_ratio=2.5)
    + facet_grid('mouse_reg~resp_type')
    #+ stat_smooth( color='red', method='lowess', size=1,span=.3)
    + stat_summary( color='red', size=1, geom='line')
    + geom_errorbar(data=data_mean, mapping=aes(x='trial',ymin='dff_resp + sem',ymax='dff_resp-sem'))
    #+ geom_jitter(size=.2,width=.15)
     # +geom_line(data=data_mean,color='r',group=1)
     )
fig=plot.draw();
fig.set_size_inches((10,2.5*len(data.mouse_reg.unique())))

In [ ]:
search = 'mouse7_region3_layer5'
viewfile = [x for x in group_files if search in x][0]
cnmf = load_CNMF(viewfile)
cnmf.estimates.dims = dims
cnmf.estimates.nb_view_components(img = None,idx=None, denoised_color='red' )

In [ ]:
plt.figure(figsize=(5,5))
plt.title('mean cell DF/F across all trials')
plt.imshow(F_dff[strong_idx].mean(axis=0).reshape((-1,trial_frames),order='C'), aspect='auto')
plt.xlabel('frame (10fps)')
plt.ylabel('trial')
plt.colorbar()


plt.figure(figsize=(5,5))
plt.title('trial avg DF/F across cells')
plt.xlabel('frame (10fps)')
plt.ylabel('cell')
plt.imshow(F_dff[strong_idx].reshape((len(strong_idx),-1,trial_frames),order='C').mean(axis=1), aspect='auto')
plt.colorbar()

# Combined Analysis

In [ ]:
# load in combined results
#combined_file = glob.glob('/gpfs/data/shohamlab/shared_data/yi_recordings/yi_new_holder_results/line3/new_experiments_201911/11132019/mouse9/region1_combined_out_p0_nb3_nocnn.hdf5')[0]
#combined_file = '/gpfs/data/shohamlab/shared_data/yi_recordings/yi_new_holder_results/sst/mouse8_framenorm_combined_out_p0_nb3_nocnn.hdf5'
#combined_file = glob.glob(os.path.join(base_dir,group_name+'*.hdf5'))[0]
combined_file = glob.glob(os.path.join(base_dir,group_dir,'*.hdf5'))[0]

print(combined_file + '\n')
comb_cnmf = load_CNMF(combined_file)
print(comb_cnmf.params)

In [ ]:

A = comb_cnmf.estimates.A
b = comb_cnmf.estimates.b
C = comb_cnmf.estimates.C
f = comb_cnmf.estimates.f
S = comb_cnmf.estimates.S
F_dff = comb_cnmf.estimates.F_dff

trial_start_offset = 100
trial_length = 100
stim_frame = 10



nframes = C.shape[1]
ntrials = (nframes - trial_start_offset)//trial_length
ncomps = C.shape[0]


In [ ]:
ncomps

In [ ]:
#mmap_file = glob.glob('/gpfs/data/shohamlab/shared_data/yi_recordings/yi_new_holder_results/line3/new_experiments_201911/11132019/mouse9/region1/*.mmap')[0]
#mmap_file = glob.glob('/gpfs/data/shohamlab/shared_data/yi_recordings/yi_new_holder_results/sst/mouse8_frame_norm/*.mmap')[0]
mmap_file = glob.glob(os.path.join(base_dir,group_dir,'*.mmap'))[0]
Yr, dims, T = cm.load_memmap(mmap_file)
print(Yr.T.shape)
images = np.reshape(Yr.T, [T] + list(dims), order='F') 
corr_img, pnr_img = cm.summary_images.correlation_pnr(images.transpose(1,2,0))

In [ ]:
comb_cnmf.estimates.dims = (256,256)
comb_cnmf.estimates.nb_view_components(img = corr_img,idx=None, denoised_color='red' )

In [ ]:
plt.figure(figsize=(12,8))
plt.subplot(1,2,1)
plt.imshow(corr_img)
plt.title('local correlations')
plt.subplot(1,2,2)
plt.imshow(pnr_img)
plt.title('peak signal-noise ratio')

In [ ]:
plt.figure(figsize=(15,7))
plt.subplot(1,3,1)
plt.imshow(b.sum(axis=1).reshape(dims,order='F'))
plt.title('background')
plt.subplot(1,3,2)
plt.imshow(A.sum(axis=-1).reshape(dims,order='F'))
plt.title('Sources')
plt.subplot(1,3,3)
plt.imshow(A.multiply((C.mean(axis=1).reshape((1,-1)))).sum(axis=-1).reshape(dims,order='F') )
plt.title('F-weighted sources')
plt.show()

In [ ]:
# examine F_dff trace in detail
xlims = slice(20000,20500)
plt.plot(F_dff[49,xlims])
plt.xticks((0,xlims.stop - xlims.start),(xlims.start,xlims.stop))
plt.show()

In [ ]:
comb_cnmf.estimates.SNR_comp

In [ ]:
stim_frame = 10
good_comps = np.where(comb_cnmf.estimates.SNR_comp > 10)[0]

plt.hist(comb_cnmf.estimates.SNR_comp)
plt.title('SNR distribution over components; {} total'.format(len(comb_cnmf.estimates.SNR_comp)))
plt.show()

In [ ]:
ax = plt.figure()
plt.plot(f.reshape((-1,100), order='C').mean(axis=0) / f.mean() -1)
plt.title('Trial-averaged background response')
ax.axes[0].axvline(x=stim_frame, color='red', linewidth=.4)
plt.xlabel('frame')
plt.ylabel('background F (% of mean)')
plt.show()

In [ ]:
group_data.name_list

In [ ]:
# Find percentiles for responses

thresh_low,thresh_high = [5,90]; # percentile thresholds
thresh_mid = [30,50] # non responder thresholds
prestim = slice(stim_frame-10,stim_frame) 
poststim = slice(stim_frame+1, stim_frame+11)



trial_mean_dff =F_dff.reshape((F_dff.shape[0],len(group_data.name_list),-1,100), order='C')[:,:,1:-1,:].mean(axis=2)
trial_mean_S = S.reshape((S.shape[0],len(group_data.name_list),-1,100), order='C')[:,:,1:-1,:].mean(axis=2)

dff_change = F_dff.reshape((F_dff.shape[0],len(group_data.name_list),-1,100),order='C')[:,:,1:-1,:]
dff_change = dff_change[:,:,:,poststim].mean(axis=-1) - dff_change[:,:,:,prestim].mean(axis=-1)
dff_change = dff_change.mean(axis=(1,2))

pct_low, pct_high = np.percentile(dff_change,(thresh_low,thresh_high))
pct_mid_low, pct_mid_high = np.percentile(dff_change,thresh_mid)
responders = np.where(dff_change>pct_high)[0]
non_responders = np.where(  (dff_change<= pct_mid_high) & (dff_change>pct_mid_low))[0]
neg_responders = np.where(dff_change<= pct_low)[0]

print('low,high df/f thresholds: {:.3f}, {:.3f}'.format( pct_low,pct_high ))
print('non-responder df/f thresholds: {:.3f}, {:3f}'.format(pct_mid_low, pct_mid_high))
print('responder counts (low,non,high): ({},{},{})'.format(len(neg_responders),len(non_responders),len(responders)))

plt.hist(dff_change, bins=20)
plt.title('Change in DF/F compared to prestim')
plt.xlabel('DF/F')
plt.ylabel('count')
plt.plot()

In [ ]:
fig = plt.figure(figsize=(6,10))
plt.imshow(trial_mean_dff[:,:,:].mean(axis=1) - trial_mean_dff[:,:,prestim].mean(axis=1).mean(axis=-1,keepdims=True))
plt.xlabel('trial frame')
plt.ylabel('component index')
plt.colorbar()
plt.title('DF/F change relative to pre-stim')
plt.show()

In [ ]:
# trial avg response traces

choose = responders
ylims = [-.05,.10]
ax = plt.figure()
tmp = trial_mean_dff[:,:,:].mean(axis=1) - trial_mean_dff[:,:,prestim].mean(axis=1).mean(axis=-1,keepdims=True)
plt.plot(tmp[choose].T)
plt.plot(tmp[choose].mean(axis=0), 'r', lw=4)
ax.axes[0].axvline(x=stim_frame, color='red', linewidth=.4)
plt.title('mean trial DF/F relative to prestim')
plt.xlabel('trial frame'); plt.ylabel('DF/F')
plt.ylim(ylims)
plt.grid()
plt.show()

In [ ]:
# Responses vs params

use_comps = neg_responders # set to good_comps to filter by high SNR, slice(None) for none
ylim=[-.05,.10]
dffs = trial_mean_dff[use_comps,:,:]
dffs = dffs[:,:,poststim].mean(axis=-1) - dffs[:,:,prestim].mean(axis=-1)
sems = dffs.std(axis=0 )/ np.sqrt(dffs.shape[0])
dffs = dffs.mean(axis=0)


tab = pd.DataFrame({'mean_resp': dffs,
                    'sem_resp': sems,
                    'mpa': group_data.mpa_list,
                    'dc': group_data.dc_list
                   })
#print(tab)



plt.figure(figsize=(14,4))


plt.subplot(1,2,1)

subtab = tab[tab.mpa==.08].sort_values('dc')
plt.errorbar(subtab.dc,subtab.mean_resp, yerr=subtab.sem_resp, capsize=3)
plt.title('post-stim response vs DC (at high amplitude)')
plt.xlabel('DC')
plt.ylabel('Trial average response (change in DF/F)')
plt.ylim(ylim)
plt.grid()

plt.subplot(1,2,2)
subtab = tab[tab.dc==.50].sort_values('mpa')
plt.errorbar(subtab.mpa,subtab.mean_resp,yerr=subtab.sem_resp, capsize=3)
plt.title('post-stim response vs amplitude (at 50% DC)')
plt.xlabel('amplitude (MPA)')
plt.ylabel('Trial average response (Change in DF/F)')
plt.ylim(ylim)
plt.grid()
plt.show()

In [ ]:
proj_traces = A.T.dot(Yr)
proj_traces_bgsub = proj_traces - A.T.dot(b).dot(f)

trial_avg_proj = proj_traces.reshape((proj_traces.shape[0],len(group_data.name_list),-1,100), order='C')[:,:,1:-1,:]
trial_avg_proj = (trial_avg_proj - trial_avg_proj[:,:,:,0:stim_frame].mean(axis=-1, keepdims=True))/ trial_avg_proj[:,:,:,0:stim_frame].mean(axis=-1, keepdims=True)
trial_avg_proj = trial_avg_proj.mean(axis=2)

trial_avg_proj_bgsub = proj_traces_bgsub.reshape((proj_traces.shape[0],len(group_data.name_list),-1,100), order='C')[:,:,1:-1]
trial_avg_proj_bgsub = (trial_avg_proj_bgsub - trial_avg_proj_bgsub[:,:,:,0:stim_frame].mean(axis=-1, keepdims=True))
trial_avg_proj_bgsub = trial_avg_proj_bgsub.mean(axis=2)

trial_avg_dffbg = f.reshape((f.shape[0],len(group_data.name_list),-1,100), order='C')[:,:,1:-1,:]
trial_avg_dffbg = (trial_avg_dffbg - trial_avg_dffbg[:,:,:,0:stim_frame].mean(axis=-1, keepdims=True))/ trial_avg_dffbg[:,:,:,0:stim_frame].mean(axis=-1, keepdims=True)
trial_sem_dffbg = trial_avg_dffbg[:,:,:,stim_frame+2:stim_frame+12].mean(axis=-1).std(axis=2) / np.sqrt(trial_avg_dffbg.shape[2])
trial_avg_dffbg = trial_avg_dffbg.mean(axis=2)



In [ ]:
np.concatenate(file_list).install

In [ ]:
choose = responders 
# choose = [24,30,33,81,90]
ylims = ylims
plt.figure(figsize=(10,4))

ax = plt.subplot(1,2,1)
plt.title('projected trial average df/f')
plt.plot(trial_avg_proj[choose].mean(axis=1).T)
plt.plot(trial_avg_proj[choose].mean(axis=(0,1)),'r', lw=4)
ax.axes.axvline(x=stim_frame, color='red', linewidth=.4)
plt.ylim(ylims)

ax = plt.subplot(1,2,2)
plt.plot(trial_avg_proj_bgsub[choose].mean(axis=1).T)
plt.plot(trial_avg_proj_bgsub[choose].mean(axis=(0,1)),'r', lw=4)
ax.axes.axvline(x=stim_frame, color='red', linewidth=.4)


plt.show()

In [ ]:
plt.imshow(A[:,choose].sum(axis=-1).reshape(dims,order='F'))

In [ ]:
trial_mean_dff.shape

In [ ]:
choose = responders

plt.figure(figsize=(14,5))

plt.subplot(1,2,1)
frame0_rel_dff = trial_mean_dff[choose,:,:4].mean(axis=(1,2)) - trial_mean_dff[choose,:,prestim].mean(axis=(1,2))
post_stim_dff = dff_change[choose]
plt.scatter(post_stim_dff, frame0_rel_dff, s=3)
plt.xlabel('dff change post-pre stim')
plt.ylabel('dff offset in early pre stim')

plt.title('caiman df/f early frame/post stim correlation : {:.2f}'.format(np.corrcoef(post_stim_dff, frame0_rel_dff)[0,1]))


plt.subplot(1,2,2)
frame0_rel_dff = trial_avg_proj[choose,:,:4].mean(axis=(1,2))
post_stim_dff = trial_avg_proj[choose,:,poststim].mean(axis=(1,2))
plt.scatter(post_stim_dff, frame0_rel_dff,s=3)
plt.title('projected df/f early frame/post stim correlation : {:.2f}'.format( np.corrcoef(post_stim_dff, frame0_rel_dff )[0,1]))

plt.subplots_adjust(wspace=.5)
plt.plot()

In [ ]:
# BACKGROUND TRIAL STATS

use_comp = 2
trial_shape_bg = f.reshape((f.shape[0],f.shape[1]//1600,-1,100), order='C')[:,:,1:-1,:]
trial_dffs = (trial_shape_bg[use_comp,:,:,stim_frame+1:stim_frame+6].mean(axis=-1)  - trial_shape_bg[use_comp,:,:,stim_frame-6:stim_frame-1].mean(axis=-1) )/ trial_shape_bg[use_comp,:,:,stim_frame-6:stim_frame-1].mean(axis=-1) 
trial_mean_dffs = trial_dffs.mean(axis=-1)
trial_sem_dffs = trial_dffs.std(axis=-1) / np.sqrt(trial_dffs.shape[-1])

tab = pd.DataFrame({'mean_resp': trial_mean_dffs,
                    'sem_resp': trial_sem_dffs,
                    'mpa': group_data.mpa_list,
                    'dc': group_data.dc_list
                   })


print(tab)



plt.figure(figsize=(14,4))

plt.subplot(1,2,1)
subtab = tab[tab.mpa==.08].sort_values('dc')
plt.errorbar(subtab.dc,subtab.mean_resp, yerr=subtab.sem_resp, capsize=3)
plt.title('post-stim response vs DC (at high amplitude)')
plt.xlabel('DC')
plt.ylabel('Trial average response (change in DF/F)')

plt.subplot(1,2,2)
subtab = tab[tab.dc==.50].sort_values('mpa')
plt.errorbar(subtab.mpa,subtab.mean_resp,yerr=subtab.sem_resp, capsize=3)
plt.title('post-stim response vs amplitude (at 50% DC)')
plt.xlabel('amplitude (MPA)')
plt.ylabel('Trial average response (change in DF/F)')
plt.show()

In [ ]:


nbg = trial_shape_bg.shape[0]

choose = slice(None)
bg_dffs = (trial_shape_bg.mean(axis=(1,2))  - trial_shape_bg[:,:,:,:stim_frame].mean(axis=-1,keepdims=True).mean(axis=(1,2)) )/ (trial_shape_bg[:,:,:,:stim_frame].mean(axis=-1,keepdims=True).mean(axis=(1,2))) 


plt.figure(figsize=(8,3*(nbg+1)))
ax = plt.subplot(nbg+1,2,1)
plt.plot(bg_dffs[choose].T)
plt.plot(bg_dffs[choose].mean(axis=0),'r', lw=4)
ax.axes.axvline(x=stim_frame, color='red', linewidth=.4)
this_lims = ax.get_ylim()
plt.grid()

for i in range(nbg):
    ax = plt.subplot(nbg+1,2,3+2*i)
    plt.plot(bg_dffs[i])
    ax.axes.axvline(x=stim_frame, color='red', linewidth=.4)
    plt.ylim(this_lims)
    plt.grid()
    ax = plt.subplot(nbg+1,2,4+2*i)
    plt.imshow(b[:,i].reshape(dims,order='F'))

In [ ]:
choose = slice(None)

plt.figure(figsize=(14,5))

plt.subplot(1,2,1)

plt.plot(trial_mean_dff[choose].mean(axis=1).std(axis=0))
plt.title('trial frame caiman DF/F std dev across ROIs')
plt.xlabel('trial frame')
plt.ylabel('std dev of DF/F')

plt.subplot(1,2,2)

plt.plot(trial_avg_proj[choose].mean(axis=1).std(axis=0))
plt.title('trial frame projected DF/F std dev across ROIs')
plt.xlabel('trial frame')
plt.ylabel('std dev of DF/F')

In [ ]:
tmp_dff =F_dff.reshape((F_dff.shape[0],len(group_data.name_list),-1,100), order='C')[:,:,1:-1,:]
tmp_dff_all = tmp_dff.mean(axis=0)
tmp_dff_frame0 = tmp_dff_all[:,:,:4].mean(axis=-1)

In [ ]:
np.unravel_index(np.argsort(tmp_dff_frame0.flatten())[-5:], tmp_dff_frame0.shape)

In [ ]:
plt.plot(tmp_dff[:,1,5,:])
plt.plot(tmp_dff[:,1,5,:].mean(axis=0), 'r', lw=3)

In [ ]:
F_dff.shape

In [ ]:
plt.imshow(F_dff[:,500:700])

In [ ]:
plt.plot(f[:,500:700].mean(axis=0).T / f[:,500:700].mean())